In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection  import cross_val_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import confusion_matrix

from sklearn.metrics import r2_score
from math import sqrt

In [9]:
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.utils.validation import check_array
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as mae

In [10]:
train = pd.read_csv('df_train_copy')
test_bm_sales = pd.read_csv('test_bm_sales.csv')

In [11]:
#train

train = train[['Item_Identifier', 'Item_Weight', 'Item_Fat_Content',
       'Item_Visibility', 'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales', 'Years_Established']]

train_df = train.drop(['Item_Identifier','Outlet_Identifier','Outlet_Establishment_Year'],axis=1)

In [15]:
#test

test_bm_sales = test_bm_sales[['Item_Identifier', 'Item_Weight', 'Item_Fat_Content',
       'Item_Visibility', 'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type']]

test_bm_sales['Years_Established'] = 2021 - test_bm_sales['Outlet_Establishment_Year']

test_df = test_bm_sales.drop(['Item_Identifier','Outlet_Identifier','Outlet_Establishment_Year'],axis=1)

In [16]:
le = LabelEncoder()
to_encode = ['Outlet_Size','Item_Fat_Content','Item_Type','Outlet_Size','Outlet_Location_Type','Outlet_Type' ]
for i in to_encode:
    train_df[i] =le.fit_transform(train_df[i])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(le_name_mapping)

{'High': 0, 'Medium': 1, 'Small': 2}
{'Low Fat': 0, 'Regular': 1}
{'Baking Goods': 0, 'Breads': 1, 'Breakfast': 2, 'Canned': 3, 'Dairy': 4, 'Frozen Foods': 5, 'Fruits and Vegetables': 6, 'Hard Drinks': 7, 'Health and Hygiene': 8, 'Household': 9, 'Meat': 10, 'Others': 11, 'Seafood': 12, 'Snack Foods': 13, 'Soft Drinks': 14, 'Starchy Foods': 15}
{0: 0, 1: 1, 2: 2}
{'Tier 1': 0, 'Tier 2': 1, 'Tier 3': 2}
{'Grocery Store': 0, 'Supermarket Type1': 1, 'Supermarket Type2': 2, 'Supermarket Type3': 3}


In [17]:
le = LabelEncoder()
to_encode = ['Outlet_Size','Item_Fat_Content','Item_Type','Outlet_Size','Outlet_Location_Type','Outlet_Type' ]
for i in to_encode:
    test_df[i] =le.fit_transform(test_df[i])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(le_name_mapping)

{'High': 0, 'Medium': 1, 'Small': 2, nan: 3}
{'LF': 0, 'Low Fat': 1, 'Regular': 2, 'low fat': 3, 'reg': 4}
{'Baking Goods': 0, 'Breads': 1, 'Breakfast': 2, 'Canned': 3, 'Dairy': 4, 'Frozen Foods': 5, 'Fruits and Vegetables': 6, 'Hard Drinks': 7, 'Health and Hygiene': 8, 'Household': 9, 'Meat': 10, 'Others': 11, 'Seafood': 12, 'Snack Foods': 13, 'Soft Drinks': 14, 'Starchy Foods': 15}
{0: 0, 1: 1, 2: 2, 3: 3}
{'Tier 1': 0, 'Tier 2': 1, 'Tier 3': 2}
{'Grocery Store': 0, 'Supermarket Type1': 1, 'Supermarket Type2': 2, 'Supermarket Type3': 3}


In [18]:
train_df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Years_Established
0,9.30,0,0.016047,4,249.8092,1,0,1,3735.1380,22
1,5.92,1,0.019278,14,48.2692,1,2,2,443.4228,12
2,17.50,0,0.016760,10,141.6180,1,0,1,2097.2700,22
3,19.20,1,0.066132,6,182.0950,1,2,0,732.3800,23
4,8.93,0,0.066132,9,53.8614,0,2,1,994.7052,34


In [19]:
test_df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Years_Established
0,20.750,1,0.007565,13,107.8622,1,0,1,22
1,8.300,4,0.038428,4,87.3198,3,1,1,14
2,14.600,1,0.099575,11,241.7538,3,2,0,23
3,7.315,1,0.015388,13,155.0340,3,1,1,14
4,NaN,2,0.118599,4,234.2300,1,2,3,36


In [20]:
X =  train_df.loc[:, train_df.columns != 'Item_Outlet_Sales']
y =  train_df.loc[:, train_df.columns == 'Item_Outlet_Sales']

In [27]:
params = {'n_estimators' : 1000, 'max_depth': 9,
 'min_child_weight': 6,
 'eta': 0.01,
 'subsample': 1.0,
 'colsample_bytree': 1.0,
 'objective': 'reg:linear',
 'eval_metric': 'mae'}

In [28]:
model = XGBRegressor(n_estimators = 1000, max_depth = 9, min_child_weight = 6, eta = 0.01, subsample = 1.0, 
                     colsample_bytree = 1.0, objective =  'reg:linear', eval_metric = 'mae' )

In [29]:
model.fit(X,y)

[22:54:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, eta=0.01,
             eval_metric='mae', gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.00999999978,
             max_delta_step=0, max_depth=9, min_child_weight=6, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1.0,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [37]:
output = model.predict(test_df)

In [33]:
df = pd.DataFrame(output, columns = ['Pedicted_Sales'])

In [41]:
df[df['Pedicted_Sales'] < 0] = 0

,Pedicted_Sales
0,1711.763672
1,1332.526855
2,660.485229
3,2311.759033
4,5623.540039
...,...
5676,2351.637695
5677,2476.449463
5678,1696.369995
5679,3703.589600


In [36]:
df.to_csv('output_results.csv')